# Loading Files and Libraries

In [8]:
import pandas as pd
df = pd.read_csv("/kaggle/input/cleaned-csv/updated_csv (1).csv")

In [ ]:
df.head()

In [13]:
df = df[:260]

# Using Tune api and Llama 3.1 8b for generating Tags as per user prompt

In [10]:
import json
import requests

stream = False
url = "https://proxy.tune.app/chat/completions"
headers = {
    "Authorization": "sk-tune-VrtwSPTRPVmEnxcxkx0YbJD41Lt0jq69351",
    "Content-Type": "application/json",
}

In [15]:
content_data = df['Cleaned_Text']

In [16]:
content_data

0      The year was 1870. New London, Connecticut, Un...
1      BzzzHey, whats all that buzz about? Is it a co...
2      Aditi and Aarav love pooris. Halwa- poori, khe...
3      Fireflies are small winged insects that glow i...
4      Sarla liked to watch birds fly. One day, durin...
                             ...                        
255    I am Isha and this is Logu, the mason . He is ...
256    Avi stretches and jumps out of bed. Today is P...
257    It was a bright morning. The hungry myna hoppe...
258    Today , Avi is a rock. Mama calls the rock for...
259    Every evening, Anvi peers through her telescop...
Name: Cleaned_Text, Length: 260, dtype: object

In [20]:
sys_prompt = """
 You are a Library Assistant tasked with categorising children’s books. 
 Your purpose is to go through the story text and generate a json object, with the following structure.Any explanation you want 
 to give can go in the explanation variable. There should be no other output and do not iterate. There should be only one json object.

{ "story_summary": "A young woman struggles to survive in a world overrun by mythical creatures.", "category": ["fantasy", "adventure"], "themes": ["friendship", "growing up", "robotics", "addition", "science"], "story_elements": ["girl protagonist", "magical creatures"], 
"labels": ["fantasy", "adventure", "friendship", "growing up", "robotics", "addition", "science", "girl protagonist", "magical creatures"] 
, "explanation": explanation"}

Here are the instructions to reason your way to the final json response. 

Step 1: Summarize the Story

Read each story carefully and summarize in one or two sentences that capture the core narrative and main characters.
Do not generate the story title

Step 2: Identify All Potential Labels

Identify this list of categories, themes and story elements that are relevant to this story including those that are secondary or peripheral.
Refer to the master corpus to see if you have missed anything.


Step 3: Evaluate Theme Centrality

For each theme identified, evaluate its centrality by answering the following questions:
Does this theme drive the main conflict or resolution of the story?
Does this theme appear throughout the story, or is it only briefly mentioned or incidental?
Is this theme connected to the primary motivation of the main characters?
Would the story lose its core meaning without this theme?

Step 4: Assign the Central Theme

From the list of themes, assign the one or two themes that meet the central theme criteria. Do not assign themes that are tangential or peripheral.


Step 5: Once you have reviewed the story and identified the central and tangential themes, combine the category, themes and story elements into a list of labels
"""

In [18]:
def generate_initial_response(content):
    payload = {
        "temperature": 0.2,
        "messages": [
            {
                "role": "system",
                "content": f"{sys_prompt}"
            },
            {
                "role": "user",
                "content": f"{content}"
            }
        ],
        "model": "meta/llama-3.1-8b-Instruct",
        "frequency_penalty": 0.2,
        "max_tokens": 1024
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        response_json = response.json()
        if 'choices' in response_json and len(response_json['choices']) > 0:
            generated_text = response_json['choices'][0]['message']['content']
            return  generated_text # Pass full content as context  
    except Exception as err:
        print(f"Error: {err}")
        return None 

In [296]:
# Assuming 'content_data' is a list or pandas DataFrame column containing the data
responses = []
from tqdm import tqdm

# Iterate through the content column and generate responses
for content in tqdm(content_data, desc="Generating Tags"):
    generated_response = generate_initial_response(content)
    if generated_response:
        responses.append({'generated_response': generated_response})

# Convert the list of responses to a pandas DataFrame
df_responses = pd.DataFrame(responses)

# Save the DataFrame to a CSV file
csv_filename = "generated_responses.csv"
df_responses.to_csv(csv_filename, index=False)

print(f"Responses saved to {csv_filename}")

Generating Tags: 100%|██████████| 111/111 [07:57<00:00,  4.30s/it]

Responses saved to generated_responses_150_262.csv


In [297]:
df_responses

generated_response
0    {\n  "story_summary": "The story explores the ...
1    {\n  "story_summary": "Bhanwari eagerly awaits...
2    {\n  "story_summary": "A young girl named Muth...
3    {\n  "story_summary": "A young girl named Mana...
4    {\n  "story_summary": "Sharanya tries to commu...
..                                                 ...
106  {\n  "story_summary": "A young boy named Avi p...
107  {\n  "story_summary": "A myna bird's plan to e...
108  {\n  "story_summary": "Avi is a child who imag...
109  {\n  "story_summary": "Anvi spots a spaceship ...
110  {\n  "story_summary": "A young transgender wom...

[111 rows x 1 columns]

In [ ]:
import json
import re

# Function to clean the JSON string by escaping invalid characters
def clean_json_string(json_str):
    # Remove invalid control characters (e.g., newlines, tabs within the JSON string)
    json_str = re.sub(r'[\x00-\x1F\x7F]', '', json_str)
    
    # Handle backslashes that are not part of escape sequences (escape any single backslashes)
    json_str = re.sub(r'\\(?![bfnrtv"\\])', r'\\\\', json_str)
    
    return json_str

# Create a new column 'Labels' in the DataFrame and initialize it as 'No Labels'
results_df['Labels'] = 'No Labels'

# Loop over each row in the 'generated_response' column
for index, response in results_df['generated_response'].items():
    if not isinstance(response, str) or len(response.strip()) == 0:
        # Skip rows with empty or non-string responses
        print(f"Skipping empty or invalid response at index {index}")
        continue
    
    try:
        # Clean the JSON string
        response_clean = clean_json_string(response)
        
        # Load the cleaned JSON string into a dictionary
        data = json.loads(response_clean)
        
        # Extract the 'labels' key from the dictionary, defaulting to 'No Labels' if any issue occurs
        labels = data.get("labels", "No Labels")
        
        # If labels are empty or invalid, set 'No Labels'
        if not labels or not isinstance(labels, list):
            labels = "No Labels"
        
        # Assign the labels to the 'Labels' column in the DataFrame at the specific index
        results_df.at[index, 'Labels'] = labels
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON at index {index}: {e}")
        results_df.at[index, 'Labels'] = "No Labels"  # Set to "No Labels" if there's a JSON error
    except Exception as e:
        print(f"Unexpected error at index {index}: {e}")
        results_df.at[index, 'Labels'] = "No Labels"  # Handle any other exceptions

In [344]:
results_df

generated_response  \
0    {\n  "story_summary": "The story is about Luci...   
1    {\n  "story_summary": "The article discusses t...   
2    {\n  "story_summary": "Aditi and Aarav learn a...   
3    {\n  "story_summary": "The story explains how ...   
4    {\n  "story_summary": "Sarla learns about bird...   
..                                                 ...   
256  {\n  "story_summary": "A young boy named Avi p...   
257  {\n  "story_summary": "A myna bird's plan to e...   
258  {\n  "story_summary": "Avi is a child who imag...   
259  {\n  "story_summary": "Anvi spots a spaceship ...   
260  {\n  "story_summary": "A young transgender wom...   

                                                Labels  
0    [biography, history, science, invention, probl...  
1    [science, nature, friendship, hard work, aging...  
2    [children, science, education, curiosity, expe...  
3    [science, nature, curiosity, discovery, commun...  
4    [children's book, science, curiosity, learning...  
..                                                 ...  
256  [children's literature, daily routine, routine...  
257  [fantasy, adventure, food chain, energy, natur...  
258  [children's literature, picture book, imaginat...  
259  [science fiction, adventure, curiosity, explor...  
260  [biography, self-discovery, social justice, id...  

[261 rows x 2 columns]

In [302]:
df['labels'] = results_df['Labels']
df['generating_response'] = results_df['generated_response']

In [351]:
df

id                                              title  \
0      3566         How Does the Toothpaste Get into the Tube?   
1      3625                                  Why Do Bees Buzz?   
2      3841                          Why Does A Poori Puff Up?   
3      3905                  Why Can't We Glow Like Fireflies?   
4      4314                             How Do Aeroplanes Fly?   
..      ...                                                ...   
256  374691                               Paint Day, Plant Day   
257  375319                           How the Myna Ate the Sun   
258  378879                                      A Rock, A Dog   
259  508218                         The Case of the New Planet   
260  561819  Grace: One engineer's fight to make science ed...   

                                    story_link author / Translator  \
0      https://storyweaver.org.in/stories/3566        Veena Prasad   
1      https://storyweaver.org.in/stories/3625   Nabanita Deshmukh   
2      https://storyweaver.org.in/stories/3841       Varsha  Joshi   
3      https://storyweaver.org.in/stories/3905   Nabanita Deshmukh   
4      https://storyweaver.org.in/stories/4314      Aditi Sarawagi   
..                                         ...                 ...   
256  https://storyweaver.org.in/stories/374691      Sanjana  Kapur   
257  https://storyweaver.org.in/stories/375319         Labonie Roy   
258  https://storyweaver.org.in/stories/378879      Sanjana  Kapur   
259  https://storyweaver.org.in/stories/508218     Ananya Dasgupta   
260  https://storyweaver.org.in/stories/561819      Sayantan Datta   

      organization published_date derivation_type reading_level language  \
0    Pratham Books     2015-11-24        Original       Level 3  English   
1    Pratham Books     2016-04-13        Original       Level 3  English   
2    Pratham Books     2016-06-09        Original       Level 3  English   
3    Pratham Books     2015-12-24        Original       Level 3  English   
4    Pratham Books     2016-02-01        Original       Level 3  English   
..             ...            ...             ...           ...      ...   
256  Pratham Books     2021-12-21        Original       Level 2  English   
257  Pratham Books     2021-12-22        Original       Level 3  English   
258  Pratham Books     2021-12-31        Original       Level 2  English   
259  Pratham Books     2022-11-14        Original       Level 3  English   
260  Pratham Books     2023-03-13        Original       Level 4  English   

                                              category  ...  story_type  \
0                             History,STEM,Non-fiction  ...        Read   
1             STEM,Series,Non-fiction,Science & Nature  ...        Read   
2                              STEM,Series,Non-fiction  ...        Read   
3             Science & Nature,STEM,Non-fiction,Series  ...        Read   
4             Series,Non-fiction,STEM,Science & Nature  ...        Read   
..                                                 ...  ...         ...   
256                          STEM,Me and My Daily Life  ...        Read   
257               Animal Stories,Science & Nature,STEM  ...        Read   
258  STEM,Spotting Books,Me and My Daily Life,My Su...  ...        Read   
259                              Science & Nature,STEM  ...        Read   
260                    Events in Life,Biographies,STEM  ...        Read   

     recommended_type  reads  \
0         Recommended  15712   
1         Recommended   9158   
2         Recommended  14537   
3         Recommended  10375   
4         Recommended  14190   
..                ...    ...   
256       Recommended  13342   
257       Recommended   5529   
258       Recommended   2731   
259       Recommended   2931   
260       Recommended   3525   

                                       tags_associated  \
0    MiddleSchoolers4, STEM, MiddleSchoolers4, STEM...   
1    buzz, nature, environment, STEM, STEM, environ...   
2 

In [ ]:
import numpy as np
df["prompt"] = df.apply(lambda row: {
    "Title": row["title"] if pd.notna(row["title"]) else "",
    "Category": row["category"] if pd.notna(row["category"]) else "",
    "Labels": row["labels"] if pd.notna(row["cleaned_tags_list"]) else "",
    "Read Level": row["reading_level"] if pd.notna(row["reading_level"]) else "",
    "Author": row["author / Translator"] if pd.notna(row["author / Translator"]) else "",
    "Hyperlink": row["story_link"] if pd.notna(row["story_link"]) else "",
    "Story": row["Cleaned_Text"] if pd.notna(row["Cleaned_Text"]) else "",
}, axis=1)

In [375]:
df.head()

id                                       title  \
0  3566  How Does the Toothpaste Get into the Tube?   
1  3625                           Why Do Bees Buzz?   
2  3841                   Why Does A Poori Puff Up?   
3  3905           Why Can't We Glow Like Fireflies?   
4  4314                      How Do Aeroplanes Fly?   

                                story_link author / Translator   organization  \
0  https://storyweaver.org.in/stories/3566        Veena Prasad  Pratham Books   
1  https://storyweaver.org.in/stories/3625   Nabanita Deshmukh  Pratham Books   
2  https://storyweaver.org.in/stories/3841       Varsha  Joshi  Pratham Books   
3  https://storyweaver.org.in/stories/3905   Nabanita Deshmukh  Pratham Books   
4  https://storyweaver.org.in/stories/4314      Aditi Sarawagi  Pratham Books   

  published_date derivation_type reading_level language  \
0     2015-11-24        Original       Level 3  English   
1     2016-04-13        Original       Level 3  English   
2     2016-06-09        Original       Level 3  English   
3     2015-12-24        Original       Level 3  English   
4     2016-02-01        Original       Level 3  English   

                                   category  ...  recommended_type  reads  \
0                  History,STEM,Non-fiction  ...       Recommended  15712   
1  STEM,Series,Non-fiction,Science & Nature  ...       Recommended   9158   
2                   STEM,Series,Non-fiction  ...       Recommended  14537   
3  Science & Nature,STEM,Non-fiction,Series  ...       Recommended  10375   
4  Series,Non-fiction,STEM,Science & Nature  ...       Recommended  14190   

                                     tags_associated  \
0  MiddleSchoolers4, STEM, MiddleSchoolers4, STEM...   
1  buzz, nature, environment, STEM, STEM, environ...   
2  Puff, MiddleSchoolers4, dough, cooking, STEM, ...   
3  MiddleSchoolers4, STEM, STEM, STEM, MiddleScho...   
4  STEM, MiddleSchoolers4, STEM, MiddleSchoolers4...   

                                               story  \
0  The year was 1870. New London, Connecticut, Un...   
1  Bzzz…Hey, what’s all that buzz about? \nIs it ...   
2  Aditi and Aarav love pooris.\nHalwa- poori, kh...   
3  Fireflies are small winged insects that glow i...   
4  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSarla like...   

                                        Cleaned_Text  \
0  The year was 1870. New London, Connecticut, Un...   
1  BzzzHey, whats all that buzz about? Is it a co...   
2  Aditi and Aarav love pooris. Halwa- poori, khe...   
3  Fireflies are small winged insects that glow i...   
4  Sarla liked to watch birds fly. One day, durin...   

                      cleaned_tags_list  \
0  toothpaste, science, problem solving   
1        {buzz,bees,nature,environment}   
2                 {poori,cooking,dough}   
3                      firefly, science   
4                    aeroplane, science   

                                      generated_tags  \
0  Here is the output in JSON format:\n\n```json\...   
1  Here's the categorization of children's book "...   
2  Here's my analysis:\n\n**Step 1: Summarize Sto...   
3  Here's my response following the specified for...   
4  Here is the output according to the Tree of Th...   

                                              labels  \
0  [biography, history, science, invention, probl...   
1  [science, nature, friendship, hard work, aging...   
2  [children, science, education, curiosity, expe...   
3  [science, nature, curiosity, discovery, commun...   
4  [children's book, science, curiosity, learning...   

                                 generating_response  \
0  {\n  "story_summary": "The story is about Luci...   
1  {\n  "story_summary": "The article discusses t...   
2  {\n  "story_summary": "Aditi and Aarav learn a...   
3  {\n  "story_summary": "The story explains how ...   
4  {\n  "story_summary": "Sarla learns about bird...   

                                              prompt  
0  {'Title': 'How Does the Toothp

In [355]:
data = pd.DataFrame(df['prompt'])

In [357]:
data.to_csv("final_cleaned.csv",index = False)

# Generating Vector Embedding

In [21]:
data = pd.read_csv("/kaggle/input/final-csv/final_cleaned.csv")

In [14]:
%%capture
!pip install sentence-transformers faiss-gpu

In [24]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [25]:
def load_data():

    # Convert 'prompt' column back to string format for encoding
    data['prompt_str'] = data['prompt'].apply(lambda x: json.dumps(x))

    # Initialize SentenceTransformer model
    model = SentenceTransformer('all-mpnet-base-v2')

    # Generate embeddings for prompts
    data['embeddings'] = data['prompt_str'].apply(lambda x: model.encode(x))

    # Convert embeddings to a NumPy array
    embeddings = np.array(data['embeddings'].tolist()).astype('float32')
    #norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
     
    # Initialize FAISS index
    d = embeddings.shape[1]  # Dimension of the embeddings
    index = faiss.IndexFlatL2(d)
    index.add(embeddings)  # Add embeddings to the FAISS index

    return data, index, model

In [26]:
# Load the data and FAISS index
data, index, model = load_data()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
data

prompt  \
0    {'Title': 'How Does the Toothpaste Get into th...   
1    {'Title': 'Why Do Bees Buzz?', 'Category': 'ST...   
2    {'Title': 'Why Does A Poori Puff Up?', 'Catego...   
3    {'Title': "Why Can't We Glow Like Fireflies?",...   
4    {'Title': 'How Do Aeroplanes Fly?', 'Category'...   
..                                                 ...   
256  {'Title': 'Paint Day, Plant Day', 'Category': ...   
257  {'Title': 'How the Myna Ate the Sun', 'Categor...   
258  {'Title': 'A Rock, A Dog', 'Category': 'STEM,S...   
259  {'Title': 'The Case of the New Planet', 'Categ...   
260  {'Title': "Grace: One engineer's fight to make...   

                                            prompt_str  \
0    "{'Title': 'How Does the Toothpaste Get into t...   
1    "{'Title': 'Why Do Bees Buzz?', 'Category': 'S...   
2    "{'Title': 'Why Does A Poori Puff Up?', 'Categ...   
3    "{'Title': \"Why Can't We Glow Like Fireflies?...   
4    "{'Title': 'How Do Aeroplanes Fly?', 'Category...   
..                                                 ...   
256  "{'Title': 'Paint Day, Plant Day', 'Category':...   
257  "{'Title': 'How the Myna Ate the Sun', 'Catego...   
258  "{'Title': 'A Rock, A Dog', 'Category': 'STEM,...   
259  "{'Title': 'The Case of the New Planet', 'Cate...   
260  "{'Title': \"Grace: One engineer's fight to ma...   

                                            embeddings  
0    [0.058360483, 0.0076860716, -0.0138509935, -0....  
1    [0.005065045, -0.020193486, -0.005447231, 0.00...  
2    [-0.02366041, -0.03359261, -0.026219582, 0.007...  
3    [0.040421303, -0.009421185, -0.002688741, -0.0...  
4    [0.029937083, 0.031523723, -0.027300538, 0.025...  
..                                                 ...  
256  [0.03003646, 0.009325567, -0.063451245, 0.0457...  
257  [0.024739943, 0.010721339, -0.021172624, 0.018...  
258  [0.0567931, -0.023813985, -0.06974372, 0.04304...  
259  [0.060468763, -0.039288133, -0.018584484, 0.03...  
260  [0.013437162, 0.029478244, -0.038345933, 0.019...  

[261 rows x 3 columns]

# Retrieve top N results from FAISS based on user query

In [27]:
# Retrieve top N results from FAISS based on user query
def retrieve_top_n(user_query, top_n, model, index, data):
    
    query_embedding = model.encode(user_query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_n)  # Search the FAISS index
    
    # Retrieve metadata for the top results
    results = []
    print(indices)  # Debugging line to see what indices are returned
    for i, idx in enumerate(indices[0]):
        try:
            result = data.iloc[idx]  # Get row by index
            # Check if 'prompt' is already a dict or if it's a string that needs to be parsed
            if isinstance(result['prompt'], dict):
                prompt_dict = result['prompt']  # It's already a dict
            elif isinstance(result['prompt'], str):
                # Use json.loads instead of eval for safe parsing
                prompt_dict = json.loads(result['prompt'])
            else:
                raise ValueError(f"Unsupported data type for 'prompt': {type(result['prompt'])}")

            # Compute match percentage based on distance
            distance = distances[0][i]
           # match_percentage = (1 - distance / np.max(distances)) * 100  # Scale to percentage

            result_metadata = {
                'Title': prompt_dict.get('Title', 'No Title'),
                'Author': prompt_dict.get('Author', 'No Author'),
                "Labels": prompt_dict.get('Labels', 'No Labels'),
                "Read Level": prompt_dict.get('Read Level', 'No Read level'),
                'Hyperlink': prompt_dict.get('Hyperlink', 'No Hyperlink'),
            }
            results.append(result_metadata)
        
        except KeyError as e:
            print(f"KeyError: {e} - Issue with index {idx}")
        except ValueError as ve:
            print(f"ValueError: {ve} - Check the format of 'prompt' at index {idx}")
        except Exception as e:
            print(f"Error processing result at index {idx}: {e}")
    
    return results


In [37]:
import json

# Function to retrieve top N matches

# Function to retrieve top N matches (with 5 parameters)

import ast

def retrieve_top_n(user_query, top_n, model, index, data):
    query_embedding = model.encode(user_query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_n)
    print("distances:", distances)

    results = []
    print("indices:", indices)

    for i, idx in enumerate(indices[0]):
        try:
            result = data.iloc[idx]
            prompt_content = result['prompt']

            # Check if already a dictionary
            if isinstance(prompt_content, dict):
                prompt_dict = prompt_content
            elif isinstance(prompt_content, str):
                # First try to load as JSON
                try:
                    prompt_dict = json.loads(prompt_content)
                except json.JSONDecodeError:
                    # Use ast.literal_eval as fallback for non-standard JSON
                    prompt_dict = ast.literal_eval(prompt_content)
            else:
                raise ValueError(f"Unsupported type for 'prompt': {type(prompt_content)}")
            distance = distances[0][i]
            result_metadata = {
                'Title': prompt_dict.get('Title', 'No Title'),
                'Author': prompt_dict.get('Author', 'No Author'),
                "Labels": prompt_dict.get('Labels', 'No Labels'),
                "Read Level": prompt_dict.get('Read Level', 'No Read level'),
                'Hyperlink': prompt_dict.get('Hyperlink', 'No Hyperlink'),
            }
            results.append(result_metadata)

        except Exception as e:
            print(f"Error processing idx {idx}: {e}")

    return results

In [38]:
retrieve_top_n("Angry Akku", 5, model, index, data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

distances: [[0.9193147 1.0595229 1.3154404 1.3639354 1.3713843]]
indices: [[ 37 193 175 139 127]]


[{'Title': 'Angry Akku',
  'Author': 'Vinayak Varma',
  'Labels': ["children's literature",
   'picture book',
   'anger management',
   'emotional expression',
   'parent-child relationship',
   'girl protagonist',
   'family dynamics',
   'school setting'],
  'Read Level': 'Level 2',
  'Hyperlink': 'https://storyweaver.org.in/stories/12433'},
 {'Title': 'Angry Akku',
  'Author': 'Pratham Books Team',
  'Labels': ['fantasy',
   'adventure',
   'survival',
   'friendship',
   'growing up',
   'girl protagonist',
   'magical creatures'],
  'Read Level': 'Level 1',
  'Hyperlink': 'https://storyweaver.org.in/stories/173986'},
 {'Title': 'Junuka and the Rice Rakshasas',
  'Author': 'Priyadarshini Gogoi,Preeti Hatibarua',
  'Labels': ['fantasy',
   'adventure',
   'friendship',
   'helping others',
   'overcoming fears',
   'girl protagonist',
   'magical creatures',
   'farming'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/130381'},
 {'Title': 'Uh-oh',
  

In [39]:
retrieve_top_n("I need stories that explain environmental science or climate change for young learners.", 5, model, index, data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

distances: [[1.0873429 1.0925512 1.0974857 1.0982503 1.0987773]]
indices: [[ 17 207 130  98  54]]


[{'Title': "At Least I'm Okay!",
  'Author': 'Alison Byrnes',
  'Labels': ["children's literature",
   'environmental fiction',
   'friendship',
   'adaptation',
   'climate change',
   'community',
   'self-discovery',
   'mountain goat protagonist',
   'animal migration',
   'ecosystem disruption'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/5398'},
 {'Title': 'Our Beautiful World',
  'Author': 'Bibek Bhattacharya',
  'Labels': ['climate change',
   'sustainability',
   'human impact on the environment',
   'non-fiction',
   'environmental',
   'historical context',
   'scientific explanations',
   'predictions'],
  'Read Level': 'Level 4',
  'Hyperlink': 'https://storyweaver.org.in/stories/266069'},
 {'Title': 'A Shade of GREEN',
  'Author': 'Rya Jetha',
  'Labels': ['science fiction',
   'adventure',
   'friendship',
   'innovation',
   'environmentalism',
   'science',
   'girl protagonists',
   'artificial intelligence',
   'pollution',
   'sust

In [40]:
retrieve_top_n("Can you recommend stories about scientific discoveries that inspire curiosity in kids?", 5, model, index, data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

distances: [[0.91088307 0.9526811  1.0142026  1.0195538  1.0303875 ]]
indices: [[177  54  79  42 103]]


[{'Title': 'The Science Complex',
  'Author': 'Abhimanyu Ghimiray',
  'Labels': ['education',
   'adventure',
   'teamwork',
   'problem-solving',
   'science',
   'classroom setting',
   'science equipment',
   'children'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/136566'},
 {'Title': "Anna's Extraordinary Experiments with Weather",
  'Author': 'Nandita Jayaraj',
  'Labels': ['biography',
   'adventure',
   'friendship',
   'growing up',
   'science',
   'curiosity',
   'perseverance',
   'girl protagonist',
   'family',
   'scientist',
   'experiments',
   'gadgets'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/28270'},
 {'Title': 'Sailing Ships and Sinking Spoons',
  'Author': 'Jamyang Gyaltsen',
  'Labels': ['children',
   'education',
   'science',
   'friendship',
   'curiosity',
   'learning',
   'young protagonists',
   'science experiment',
   'water',
   'buoyancy',
   'displacement'],
  'Read Level': 'Lev

In [41]:
retrieve_top_n("What are some engaging stories that teach about space exploration or astronauts?", 5, model, index, data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

distances: [[0.90988594 0.9904442  1.0122634  1.1087903  1.2390513 ]]
indices: [[ 42  28  63  59 260]]


[{'Title': '3... 2... 1... Blast-off!',
  'Author': 'Sheela Preuitt,Praba Ram',
  'Labels': ['science',
   'education',
   'curiosity',
   'exploration',
   'space',
   'stars',
   'planets',
   'satellites',
   'comets',
   'telescopes',
   'dwarf planets',
   'rockets',
   'astronauts'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/25462'},
 {'Title': 'Gul in Space',
  'Author': 'Richa  Jha',
  'Labels': ["children's literature",
   'science fiction',
   'adventure',
   'exploration',
   'friendship',
   'growing up',
   'science',
   'girl protagonist',
   'space travel',
   'weightlessness',
   'space station',
   'astronaut'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/11050'},
 {'Title': 'Panipuri inside a Spaceship',
  'Author': 'Vidya Pradhan',
  'Labels': ['children',
   'science',
   'adventure',
   'friendship',
   'growing up',
   'science',
   'curiosity',
   'exploration',
   'girl protagonist',
   'famil

In [42]:
retrieve_top_n("Can you suggest a few STEM stories set in rural settings?", 5, model, index, data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

distances: [[1.0858663 1.1353097 1.1904604 1.2226832 1.2371864]]
indices: [[ 91 220 260  87 177]]


[{'Title': 'The Seed Savers',
  'Author': 'Bijal Vachharajani',
  'Labels': ["children's literature",
   'social responsibility',
   'environmentalism',
   'community',
   'sustainability',
   'empowerment',
   'cooperation',
   'problem-solving',
   'girl protagonists',
   'village setting',
   'seed bank concept'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/41319'},
 {'Title': 'The Plant Whisperer',
  'Author': 'Sayantan Datta',
  'Labels': ['biography',
   'science',
   'friendship',
   'growing up',
   'overcoming obstacles',
   'self-discovery',
   'altruism',
   'girl protagonist',
   'plant protagonist',
   'scientific research',
   'overcoming racism and sexism'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/342422'},
 {'Title': "Grace: One engineer's fight to make science education accessible for all",
  'Author': 'Sayantan Datta',
  'Labels': ['biography',
   'self-discovery',
   'social justice',
   'identit

In [43]:
retrieve_top_n("Can you suggest a stories with children cooking in the kitchen?", 5, model, index, data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

distances: [[0.70757383 0.9092118  0.91846186 0.9976262  1.0112835 ]]
indices: [[245   8  83  61 112]]


[{'Title': 'Chop Chop Sizzle',
  'Author': 'Sanjana  Kapur',
  'Labels': ["children's book",
   'cookbook',
   'family',
   'cooperation',
   'patterns',
   'helping others',
   'child protagonist',
   'family cooking together',
   'patterns in everyday life'],
  'Read Level': 'Level 2',
  'Hyperlink': 'https://storyweaver.org.in/stories/366114'},
 {'Title': 'Dum Dum-a-Dum Biryani!',
  'Author': 'Gayathri Tirthapura',
  'Labels': ["children's literature",
   'food',
   'family',
   'sibling relationship',
   'cooperation',
   'problem-solving',
   'mathematics',
   'child protagonist',
   'sister character',
   'food preparation',
   'mathematical calculations'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/4772'},
 {'Title': 'A Whistling Good Idea',
  'Author': 'Rahul Raghavan',
  'Labels': ['children',
   'science',
   'creativity',
   'problem-solving',
   'science',
   'engineering',
   'girl protagonist',
   'machine building',
   'cause-and-effect

In [44]:
retrieve_top_n("Can you suggest for a nauthy child?", 5, model, index, data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

distances: [[1.2343081 1.2378066 1.2454153 1.2516855 1.2726226]]
indices: [[109 230 253  50   7]]


[{'Title': 'How to Solve a Problem like Himani',
  'Author': 'Mala Kumar',
  'Labels': ["children's literature",
   'problem-solving',
   'friendship',
   'community service',
   'innovation',
   'environmental awareness',
   'girl protagonist',
   'school setting',
   'community involvement',
   'problem-solving'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/52584'},
 {'Title': 'Nanammal and Kamalathal Travel by Sea',
  'Author': 'Menaka Raman',
  'Labels': ["children's literature",
   'adventure',
   'friendship',
   'adventure',
   'exploration',
   'problem-solving',
   'girl protagonists',
   'birdwatching',
   'travel',
   'transportation'],
  'Read Level': 'Level 2',
  'Hyperlink': 'https://storyweaver.org.in/stories/353163'},
 {'Title': 'Nanammal and Kamalathal Travel by Air',
  'Author': 'Menaka Raman',
  'Labels': ["children's literature",
   'friendship',
   'adventure',
   'girl protagonist',
   'best friends',
   'air transport',
   'cloud

In [46]:
retrieve_top_n("Do you have stories going to doctors?", 5, model, index, data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

distances: [[1.405373  1.526962  1.5356226 1.5667423 1.6194937]]
indices: [[ 57 167   0 204  41]]


[{'Title': "Mukhil's Bloody Adventure",
  'Author': 'Kannal  Achuthan',
  'Labels': ["children's literature",
   'health',
   'bravery',
   'fear',
   'science',
   'child protagonist',
   'medical procedure',
   'mother-child relationship'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/28890'},
 {'Title': 'Ira Investigates the Invisible',
  'Author': 'Menaka Raman',
  'Labels': ['science',
   'education',
   'curiosity',
   'perseverance',
   'friendship',
   'discovery',
   'girl protagonist',
   'microbiologist',
   'microorganisms',
   'experiment'],
  'Read Level': 'Level 4',
  'Hyperlink': 'https://storyweaver.org.in/stories/123858'},
 {'Title': 'How Does the Toothpaste Get into the Tube?',
  'Author': 'Veena Prasad',
  'Labels': ['biography',
   'history',
   'science',
   'invention',
   'problem-solving',
   'innovation',
   'health',
   'boy protagonist',
   'father figure',
   'dentist',
   'toothpaste',
   'metal tubes'],
  'Read Level': 'Le

In [377]:
retrieve_top_n("I’m looking for stories that explore technological advancements and their impact on society.", 5, model, index, data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[102   9 121  83 260]]


[{'Title': 'A Stitch in Time',
  'Author': 'Veena Prasad,Himadri  Das',
  'Labels': ['children',
   'education',
   'environment',
   'conservation',
   'reduction',
   'recycling',
   'repair',
   'sustainability',
   'child protagonist',
   'friendship',
   'problem-solving',
   'science'],
  'Read Level': 'Level 4',
  'Hyperlink': 'https://storyweaver.org.in/stories/49732',
  'Match_Percentage': '1.18'},
 {'Title': 'Bonda and Devi',
  'Author': 'Roopa Pai',
  'Labels': ['science fiction',
   'adventure',
   'friendship',
   'technology',
   'growth',
   'self-discovery',
   'robot protagonist',
   'girl protagonist',
   'advanced technology'],
  'Read Level': 'Level 4',
  'Hyperlink': 'https://storyweaver.org.in/stories/4782',
  'Match_Percentage': '1.28'},
 {'Title': 'Cyborg Dadu',
  'Author': 'Lavanya Karthik',
  'Labels': ['science',
   'technology',
   'health',
   'innovation',
   'cyborg',
   'bionics',
   'artificial body parts',
   'human-machine interaction'],
  'Read Level

# Querying using indic Language

In [ ]:
pip install googletrans==4.0.0-rc1

In [50]:
from googletrans import Translator, LANGUAGES

# Initialize the Google Translator
translator = Translator()

def retrieve_top_n(user_query, top_n, model, index, data):
    if user_query:
        # Detect the language of the input
        detected_language = translator.detect(user_query).lang

        if detected_language != 'en':
            # Translate to English
            translation = translator.translate(user_query, src=detected_language, dest='en')
            user_query = translation.text
            print(f"Translated Query: {user_query} (from {LANGUAGES[detected_language]})")  # Replace st.success with print
        query_embedding = model.encode(user_query).reshape(1, -1)
        distances, indices = index.search(query_embedding, top_n)
        print("distances:", distances)

        results = []
        print("indices:", indices)

    for i, idx in enumerate(indices[0]):
        try:
            result = data.iloc[idx]
            prompt_content = result['prompt']

            # Check if already a dictionary
            if isinstance(prompt_content, dict):
                prompt_dict = prompt_content
            elif isinstance(prompt_content, str):
                # First try to load as JSON
                try:
                    prompt_dict = json.loads(prompt_content)
                except json.JSONDecodeError:
                    # Use ast.literal_eval as fallback for non-standard JSON
                    prompt_dict = ast.literal_eval(prompt_content)
            else:
                raise ValueError(f"Unsupported type for 'prompt': {type(prompt_content)}")
            distance = distances[0][i]
            result_metadata = {
                'Title': prompt_dict.get('Title', 'No Title'),
                'Author': prompt_dict.get('Author', 'No Author'),
                "Labels": prompt_dict.get('Labels', 'No Labels'),
                "Read Level": prompt_dict.get('Read Level', 'No Read level'),
                'Hyperlink': prompt_dict.get('Hyperlink', 'No Hyperlink'),
            }
            results.append(result_metadata)

        except Exception as e:
            print(f"Error processing idx {idx}: {e}")

    return results

In [51]:
retrieve_top_n("मैं ऐसी कहानियों की तलाश में हूं जो तकनीकी प्रगति और समाज पर उनके प्रभाव का पता लगाती हों", 5, model, index, data)

Translated Query: I am looking for stories that find out technological progress and their influence on society (from hindi)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

distances: [[1.153361  1.256887  1.271581  1.2825099 1.3001232]]
indices: [[102 260  83  77 121]]


[{'Title': 'A Stitch in Time',
  'Author': 'Veena Prasad,Himadri  Das',
  'Labels': ['children',
   'education',
   'environment',
   'conservation',
   'reduction',
   'recycling',
   'repair',
   'sustainability',
   'child protagonist',
   'friendship',
   'problem-solving',
   'science'],
  'Read Level': 'Level 4',
  'Hyperlink': 'https://storyweaver.org.in/stories/49732'},
 {'Title': "Grace: One engineer's fight to make science education accessible for all",
  'Author': 'Sayantan Datta',
  'Labels': ['biography',
   'self-discovery',
   'social justice',
   'identity',
   'acceptance',
   'empowerment',
   'education',
   'social justice',
   'transgender protagonist',
   'Dalit protagonist',
   'female protagonist',
   'science education',
   'activism'],
  'Read Level': 'Level 4',
  'Hyperlink': 'https://storyweaver.org.in/stories/561819'},
 {'Title': 'A Whistling Good Idea',
  'Author': 'Rahul Raghavan',
  'Labels': ['children',
   'science',
   'creativity',
   'problem-solvin

In [52]:
retrieve_top_n("మీకు వైద్యుల వద్దకు వెళ్లే కథనాలు ఉన్నాయా?", 5, model, index, data)

Translated Query: Do you have articles that go to doctors? (from telugu)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

distances: [[1.2916994 1.3143893 1.3888457 1.4297291 1.431776 ]]
indices: [[ 57 167 121 182 204]]


[{'Title': "Mukhil's Bloody Adventure",
  'Author': 'Kannal  Achuthan',
  'Labels': ["children's literature",
   'health',
   'bravery',
   'fear',
   'science',
   'child protagonist',
   'medical procedure',
   'mother-child relationship'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/28890'},
 {'Title': 'Ira Investigates the Invisible',
  'Author': 'Menaka Raman',
  'Labels': ['science',
   'education',
   'curiosity',
   'perseverance',
   'friendship',
   'discovery',
   'girl protagonist',
   'microbiologist',
   'microorganisms',
   'experiment'],
  'Read Level': 'Level 4',
  'Hyperlink': 'https://storyweaver.org.in/stories/123858'},
 {'Title': 'Cyborg Dadu',
  'Author': 'Lavanya Karthik',
  'Labels': ['science',
   'technology',
   'health',
   'innovation',
   'cyborg',
   'bionics',
   'artificial body parts',
   'human-machine interaction'],
  'Read Level': 'Level 3',
  'Hyperlink': 'https://storyweaver.org.in/stories/61582'},
 {'Title': 'The